# Q0 – Forecast Summary and Distribution Diagnostics

This notebook summarizes the probabilistic forecasts produced by selected models across multiple simulated data-generating processes (DGPs).

### Objective
- Evaluate the statistical properties of forecast distributions.
- Compare model-generated returns against true DGPs via KL divergence.

### Key Outputs
- 📄 Tables:  
  - Mean & standard deviation of returns  
  - Percentile summaries  
  - KL divergence against DGPs

- 📊 Plots:  
  - Forecast interval plots  
  - KDEs and CDFs of predicted returns  
  - KL comparisons (true vs. model returns)

### Notes
- Evaluated models:
  - Chronos, Lag-LLaMA, Moirai, TiReX, ToTo, TimesFM
- Targets: both **prices** and **returns**
- Selected forecast days: `Day 2`, `Day 12`, `Day 22`
- Context lengths vary per run


In [1]:
# Packages
import pickle
import numpy as np
import pandas as pd
from pathlib import Path

from utils.evaluation import (
    compute_kl_divergence,
    format_pivot_table,
    dataframe_to_latex
)

from utils.plotting import (
    plot_forecast,
    plot_daily_return_kdes,
    plot_daily_return_cdfs,
    plot_multiple_return_kde_comparison,
)

### Models List

Models can be added or removed from the followng list.

In [2]:
# Selected Models for Analysis
selected_model_names = [
    "chronos_model_tiny",
    "chronos_model_mini",
    "chronos_model_base",
    "lag_llama_model",
    "moirai_model_small",
    "moirai_model_base",
    "toto_model",
    "tirex_model",
    "timesfm_model_small",
    "timesfm_model_large"
]

In [3]:
# Paths and Setup
results_dir = Path("results_q0_summary")
tables_dir = results_dir / "tables"
plots_forecast_dir = results_dir / "plots_forecast"
plots_kdes_dir = results_dir / "plots_kdes"
plots_cdfs_dir = results_dir / "plots_cdfs"
plots_kl_dir = results_dir / "plots_kl"

for folder in [tables_dir, plots_forecast_dir, plots_kdes_dir, plots_cdfs_dir, plots_kl_dir]:
    folder.mkdir(parents=True, exist_ok=True)

forecast_dir = Path("forecasts")
run_dir = Path("runfiles")
datasets_dir = Path("datasets")

selected_days = [0, 10, 20]
ordered_days = [f"Day {d+2}" for d in selected_days]
ordered_percentiles = ["p1%", "p5%", "p25%", "p50%", "p75%", "p95%", "p99%"]
percentile_values = [int(p.replace("p", "").replace("%", "")) for p in ordered_percentiles]
dgp_types_kl = ["gbm_low_vol", "gbm_high_vol", "t_garch", "mixture_normal", "seasonal"]

### Loading the Forecasts

We load the forecasts and retrieve the specifics.

In [4]:
# Load Forecasts
forecast_files = sorted(forecast_dir.glob("forecast_*.pkl"))
results = []

for forecast_file in forecast_files:
    run_name = forecast_file.stem
    run_file = run_dir / f"{run_name}.txt"
    if not run_file.exists():
        continue

    run_config = {}
    with open(run_file, "r") as f:
        for line in f:
            if "=" in line:
                key, value = [x.strip() for x in line.strip().split("=", 1)]
                try:
                    run_config[key] = eval(value)
                except:
                    run_config[key] = value.strip("\"'").strip("'")

    try:
        with open(forecast_file, "rb") as f:
            forecast_result = pickle.load(f)
            low, median, high, samples, base_price = forecast_result
    except Exception:
        continue

    results.append({
        "run_name": run_name,
        "model_name": run_config["model_name"],
        "dgp_type": run_config["dataset_name"],
        "target_type": run_config["target_type"],
        "context_length": run_config["context_length"],
        "samples": samples,
        "low": low,
        "median": median,
        "high": high,
        "base_price": base_price
    })

# Filter Results by Selected Models
results = [r for r in results if r["model_name"] in selected_model_names]

In [5]:
# Split by Target Type
price_results = [r for r in results if r["target_type"] == "prices"]
return_results = [r for r in results if r["target_type"] == "returns"]

### Defining Functions

We define 2 new special functions to save tables and compute the KL divergence compatible with this notebook setup.

In [6]:
# Forecast Summary and Percentiles
def process_and_save_tables(results_subset, label):
    summary_rows = []
    percentile_rows = []

    for item in results_subset:
        is_price = item["target_type"] == "prices"
        returns = item["samples"]
        if is_price:
            returns = returns[:, 1:] / returns[:, :-1] - 1

        for day in selected_days:
            daily = returns[:, day]
            summary_rows.append({
                "context_length": item["context_length"],
                "dgp_type": item["dgp_type"],
                "model_name": item["model_name"],
                "day": f"Day {day+2}",
                "mean_return (%)": np.mean(daily) * 100,
                "std_return (%)": np.std(daily) * 100
            })

            for p, v in zip(ordered_percentiles, np.percentile(daily, percentile_values)):
                percentile_rows.append({
                    "context_length": item["context_length"],
                    "dgp_type": item["dgp_type"],
                    "model_name": item["model_name"],
                    "day": f"Day {day+2}",
                    "percentile": p,
                    "return (%)": v * 100
                })

    df_summary = pd.DataFrame(summary_rows).round(2)
    df_percent = pd.DataFrame(percentile_rows).round(2)

    pivot_summary = df_summary.pivot_table(
        index=["context_length", "dgp_type", "model_name"],
        columns="day",
        values=["mean_return (%)", "std_return (%)"]
    )
    dataframe_to_latex(format_pivot_table(pivot_summary, selected_days), tables_dir / f"forecast_table_{label}.tex")

    pivot_percent = df_percent.pivot_table(
        index=["context_length", "dgp_type", "model_name"],
        columns=["day", "percentile"],
        values="return (%)"
    )
    ordered_columns = pd.MultiIndex.from_product([ordered_days, ordered_percentiles])
    pivot_percent = pivot_percent.reindex(columns=ordered_columns)
    dataframe_to_latex(format_pivot_table(pivot_percent, selected_days), tables_dir / f"percentiles_table_{label}.tex")

In [7]:
# KL Divergence Table
def compute_and_save_kl(results_subset, label):
    rows = []
    for item in results_subset:
        if item["dgp_type"] not in dgp_types_kl:
            continue

        is_price = item["target_type"] == "prices"
        model_returns = item["samples"]
        if is_price:
            model_returns = model_returns[:, 1:] / model_returns[:, :-1] - 1

        dgp_path = datasets_dir / f"{item['dgp_type']}_returns_paths.npy"
        if not dgp_path.exists():
            continue

        dgp_returns = np.load(dgp_path)

        for day_index in selected_days:
            try:
                p = dgp_returns[:, day_index]
                q = model_returns[:, day_index]
                kl = compute_kl_divergence(p, q)
                rows.append({
                    "context_length": item["context_length"],
                    "dgp_type": item["dgp_type"],
                    "model_name": item["model_name"],
                    "day": f"Day {day_index+2}",
                    "kl_divergence": kl
                })
            except:
                continue

    df_kl = pd.DataFrame(rows).round(4)
    filename = f"kl_divergence_table_{label}.tex"
    pivot_kl = df_kl.pivot_table(
        index=["context_length", "dgp_type", "model_name"],
        columns="day",
        values="kl_divergence"
    )
    dataframe_to_latex(format_pivot_table(pivot_kl, selected_days), tables_dir / filename)
    return df_kl

In [8]:
# Execute Table Generation
process_and_save_tables(price_results, "prices")
process_and_save_tables(return_results, "returns")

df_kl_prices = compute_and_save_kl(price_results, "prices")
df_kl_returns = compute_and_save_kl(return_results, "returns")

### Plotting

We plot in a loop below.

In [9]:
# Forecast, KDE, CDF, and KL Comparison Plots
for item in results:
    run_id = item["run_name"].replace("forecast_", "")
    context_length = item["context_length"]
    dgp_type = item["dgp_type"]
    is_price = item["target_type"] == "prices"

    series_file = datasets_dir / f"{dgp_type}_{item['target_type']}.csv"
    if not series_file.exists():
        continue

    series = pd.read_csv(series_file).squeeze()
    series.index = range(len(series))

    plot_forecast(
        series=series,
        low=item["low"],
        median=item["median"],
        high=item["high"],
        dgp_type=dgp_type,
        context_length=context_length,
        path=str(plots_forecast_dir / f"forecast_{run_id}.png"),
        is_price_data=is_price
    )

    plot_daily_return_kdes(
        samples=item["samples"],
        selected_days=selected_days,
        dgp_type=dgp_type,
        context_length=context_length,
        path=str(plots_kdes_dir / f"kdes_{run_id}.png"),
        is_price_data=is_price
    )

    plot_daily_return_cdfs(
        samples=item["samples"],
        selected_days=selected_days,
        dgp_type=dgp_type,
        context_length=context_length,
        path=str(plots_cdfs_dir / f"cdfs_{run_id}.png"),
        is_price_data=is_price
    )

    if dgp_type in dgp_types_kl:
        dgp_path = datasets_dir / f"{dgp_type}_returns_paths.npy"
        if not dgp_path.exists():
            continue

        dgp_returns = np.load(dgp_path)
        model_samples = item["samples"]
        model_returns = model_samples[:, 1:] / model_samples[:, :-1] - 1 if is_price else model_samples

        plot_multiple_return_kde_comparison(
            dgp_samples=dgp_returns,
            model_samples=model_returns,
            selected_days=selected_days,
            dgp_type=dgp_type,
            context_length=context_length,
            path=str(plots_kl_dir / f"kl_{run_id}.png"),
            is_price_data=False
        )